# Test Metrics

This notebook is used for metrics measurements, about the different tasks involved in the image reconstruction pipeline.

## Image RAG

### Fixture

In [ ]:
chroma_client = chromadb.EphemeralClient()  #By default, we use an in-memory approach which does not persist anything for this test.
collection = chroma_client.create_collection(name="eustachian_collection")

In [ ]:
#TODO: Insert automatic image insertion, using jsonl file to couple each one to a caption.

## Image Editing

In [ ]:
def generate_image(inputs, output_filename = "output_edit"):
    with torch.inference_mode():
        output = pipeline(**inputs)
        i = 0
        for output_image in output.images:
            output_image.save(f"{output_filename}_{i}.jpeg")
            i += 1
        print("Image generated (saved!)")
    return output_image

def make_inputs(images, prompt: str, guidance = 4.0, neg_prompt = " ", inf_steps = 50, gen_images = 1):
    return {
    "image": images,
    "prompt": prompt,
    "generator": torch.manual_seed(0),
    "true_cfg_scale": guidance,
    "negative_prompt": neg_prompt,
    "num_inference_steps": inf_steps,
    "num_images_per_prompt": gen_images
    }


In [ ]:
def interact(prompt: str, images: List[Image.Image]):
    base64_images = []
    for image in images:
        with BytesIO() as buffered:
            image.save(buffered, format="JPEG")
            base64_images.append(base64.b64encode(buffered.getvalue()).decode("utf8"))
    response: ChatResponse = chat(model=MODEL, messages=[
    {
        'role': 'user',
        'content': prompt,
        "images": base64_images
    }
    ])
    return response['message']['content']

In [ ]:
def polish_prompt_en(original_prompt: str, images: List[Image.Image]):
    prompt = f'''
        # Edit Instruction Rewriter
        You are a professional edit instruction rewriter. Your task is to generate a precise, concise, and visually achievable professional-level edit instruction based on the user-provided instruction and the image to be edited.  

        Please strictly follow the rewriting rules below:

        ## 1. General Principles
        - Keep the rewritten prompt **concise**. Avoid overly long sentences and reduce unnecessary descriptive language.  
        - If the instruction is contradictory, vague, or unachievable, prioritize reasonable inference and correction, and supplement details when necessary.  
        - Keep the core intention of the original instruction unchanged, only enhancing its clarity, rationality, and visual feasibility.  
        - All added objects or modifications must align with the logic and style of the edited input image’s overall scene.  

        ## 2. Task Type Handling Rules
        ### 1. Add, Delete, Replace Tasks
        - If the instruction is clear (already includes task type, target entity, position, quantity, attributes), preserve the original intent and only refine the grammar.  
        - If the description is vague, supplement with minimal but sufficient details (category, color, size, orientation, position, etc.). For example:  
            > Original: "Add an animal"  
            > Rewritten: "Add a light-gray cat in the bottom-right corner, sitting and facing the camera"  
        - Remove meaningless instructions: e.g., "Add 0 objects" should be ignored or flagged as invalid.  
        - For replacement tasks, specify "Replace Y with X" and briefly describe the key visual features of X.  

        ### 2. Text Editing Tasks
        - All text content must be enclosed in English double quotes `" "`. Do not translate or alter the original language of the text, and do not change the capitalization.  
        - **For text replacement tasks, always use the fixed template:**
            - `Replace "xx" to "yy"`.  
            - `Replace the xx bounding box to "yy"`.  
        - If the user does not specify text content, infer and add concise text based on the instruction and the input image’s context. For example:  
            > Original: "Add a line of text" (poster)  
            > Rewritten: "Add text \"LIMITED EDITION\" at the top center with slight shadow"  
        - Specify text position, color, and layout in a concise way.  

        ### 3. Human Editing Tasks
        - Maintain the person’s core visual consistency (ethnicity, gender, age, hairstyle, expression, outfit, etc.).  
        - If modifying appearance (e.g., clothes, hairstyle), ensure the new element is consistent with the original style.  
        - **For expression changes, they must be natural and subtle, never exaggerated.**  
        - If deletion is not specifically emphasized, the most important subject in the original image (e.g., a person, an animal) should be preserved.
            - For background change tasks, emphasize maintaining subject consistency at first.  
        - Example:  
            > Original: "Change the person’s hat"  
            > Rewritten: "Replace the man’s hat with a dark brown beret; keep smile, short hair, and gray jacket unchanged"  

        ### 4. Style Transformation or Enhancement Tasks
        - If a style is specified, describe it concisely with key visual traits. For example:  
            > Original: "Disco style"  
            > Rewritten: "1970s disco: flashing lights, disco ball, mirrored walls, colorful tones"  
        - If the instruction says "use reference style" or "keep current style," analyze the input image, extract main features (color, composition, texture, lighting, art style), and integrate them into the prompt.  
        - **For coloring tasks, including restoring old photos, always use the fixed template:** "Restore old photograph, remove scratches, reduce noise, enhance details, high resolution, realistic, natural skin tones, clear facial features, no distortion, vintage photo restoration"  
        - If there are other changes, place the style description at the end.

        ## 3. Rationality and Logic Checks
        - Resolve contradictory instructions: e.g., "Remove all trees but keep all trees" should be logically corrected.  
        - Add missing key information: if position is unspecified, choose a reasonable area based on composition (near subject, empty space, center/edges).  

        # Output Format Example
        "211 floors high skyscrape, majesticly dominating the crowded street below..."

        Prompt to be rewritten: {original_prompt}
    '''
    return interact(prompt, images)
    

In [ ]:
#TODO: Insert random selection from images to rotate multiple times

## Image Evaluation